<a href="https://colab.research.google.com/github/shankar045/1BM23CS309_BIS/blob/main/Lab%205%20GWO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings("ignore", category=ConvergenceWarning)

data = load_breast_cancer()
X, y = data.data, data.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

n_features = X.shape[1]
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42
)

def fitness(solution):
    if np.sum(solution) == 0:
        return 0

    selected = np.where(solution == 1)[0]
    try:
        model = SVC(kernel='linear', C=1, max_iter=5000)
        model.fit(X_train[:, selected], y_train)
        preds = model.predict(X_val[:, selected])
        acc = accuracy_score(y_val, preds)
    except Exception:
        acc = 0
    return 0.9 * acc + 0.1 * (1 - (np.sum(solution) / n_features))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

N, T = 6, 10
wolves = np.random.randint(0, 2, (N, n_features))
fitness_values = np.array([fitness(w) for w in wolves])
alpha, beta, delta = wolves[np.argsort(-fitness_values)[:3]]
a = 2

for t in range(T):
    for i in range(N):
        for j in range(n_features):
            r1, r2 = np.random.rand(), np.random.rand()
            A1, C1 = 2*a*r1 - a, 2*r2
            D_alpha = abs(C1*alpha[j] - wolves[i][j])
            X1 = alpha[j] - A1*D_alpha

            r1, r2 = np.random.rand(), np.random.rand()
            A2, C2 = 2*a*r1 - a, 2*r2
            D_beta = abs(C2*beta[j] - wolves[i][j])
            X2 = beta[j] - A2*D_beta

            r1, r2 = np.random.rand(), np.random.rand()
            A3, C3 = 2*a*r1 - a, 2*r2
            D_delta = abs(C3*delta[j] - wolves[i][j])
            X3 = delta[j] - A3*D_delta

            new_value = (X1 + X2 + X3) / 3
            wolves[i][j] = 1 if sigmoid(new_value) > 0.5 else 0

    fitness_values = np.array([fitness(w) for w in wolves])
    sorted_indices = np.argsort(-fitness_values)
    alpha, beta, delta = wolves[sorted_indices[0]], wolves[sorted_indices[1]], wolves[sorted_indices[2]]
    a = 2 - 2*(t / T)

best_features = np.where(alpha == 1)[0]
best_fitness = fitness(alpha)

print(" Best Selected Feature Indices:", best_features)
print(" Best Fitness (Weighted Accuracy):", round(best_fitness, 4))



 Best Selected Feature Indices: [ 0  1  3  4  5  6  7  9 10 11 12 13 14 15 19 20 21 22 23 24 26 27 28 29]
 Best Fitness (Weighted Accuracy): 0.8989
